In [0]:
# Run Docker to Test on Port 26
# See https://mailosaur.com/blog/a-guide-to-smtp4dev

In [0]:
# -*- coding: utf-8 -*-
# Import standard libraries

import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from dataiku.customrecipe import get_recipe_config

In [0]:
# Import libraries necessary to send emails via SMTP

from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
import os
from datetime import datetime
import time # added to created delay
import random
import logging

In [0]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')

In [0]:
# Hard coded SMTP settings (for now, as of 2023-08-24)

_server = 'localhost'
_port = '26'

In [0]:
#Function to write a simple email with subject, to, sender, cc

def create_simple_mail(subject, to, sender, cc):
    mail = MIMEMultipart()
    mail['Subject'] = subject
    mail['To'] = to
    mail['From'] = sender
    mail['cc'] = cc
    return mail

In [0]:
#Function to attach a file to an email as an attachment

def attach_file_to_mail(mail,f):
    with open(f, "rb") as fil:
        part = MIMEApplication(fil.read(), Name=os.path.basename(f))
        part['Content-Disposition'] = 'attachment; filename="%s"' % os.path.basename(f)
        mail.attach(part)
    return mail

In [0]:
#Function that sends an html email and includes multiple attachments

def mail_html(subject, to, sender, cc, html, attachments=None):
    rcpt = to.split(",") + cc.split(",")
    mail = create_simple_mail(subject=subject, to=to, sender=sender, cc=cc)
    mail.attach(MIMEText(html, 'html'))
    if attachments is not None:
        for f in attachments:
            mail = attach_file_to_mail(mail,f)

    smtp_connection = smtplib.SMTP(_server, _port, timeout=120)
    with smtp_connection as s:
        s.sendmail(sender, rcpt, mail.as_string())

In [0]:
#This is a variable containing the html content to be included in the body of the email

html_text = """
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style='font-size:21px;font-family:"Arial",sans-serif;'>Ladder Plan Report&nbsp;</span><span style='font-size:19px;font-family:"Arial",sans-serif;'>&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'>&nbsp;</p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">Good Morning Team,</span><span style='font-size:19px;font-family:"Arial",sans-serif;'>&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">Attached is today&apos;s Ladder Report.&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><u><span style="font-size:16px;">This version includes <strong>Dotcom/Store data</strong></span></u><span style="font-size:16px;">, please continue to provide feedback for improvements.</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><strong><span style="font-size:16px;">How can suppliers filter to Ecomm Only Items?</span></strong></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">Item Replenishable Indicator (Column IE): filter to repln Y or N items</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">Channel Type (Column IQ): filter to WMT.com&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">Criteria for item to show:</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">Published on the site = True</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">Active = True</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">Inventory type (own or both)</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">Orderable = True</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style='font-size:19px;font-family:"Arial",sans-serif;'><a href="https://supplieracademy.wal-mart.com/mod/wiki/view.php?id=598"><strong><span style='font-size:16px;font-family:"Calibri",sans-serif;'>Training video Available&nbsp;</span></strong></a></span><strong><span style="font-size:16px;">When you click it, it will prompt you to log into Retail Link and then take you directly to the video.&nbsp;</span></strong></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><strong><span style="font-size:16px;">Problems accessing the video? Follow this path!</span></strong></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><strong><span style="font-size:16px;">Retail Link &gt; Academy &gt; Walmart &gt; Replenishment &gt; Ladder Planning</span></strong></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><strong><span style="font-size:16px;">Supplier Receipts is what is landing at your facility.&nbsp;</span></strong></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><strong><span style="font-size:16px;">Supplier Flow is what you are flowing to Walmart. This will then project your on hands based on those qty&apos;s.</span></strong></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">To use the Supplier ladder plan file, just copy your data into the Ladder plan file on the data drop tab, and on the compile tab hit Build Sheet.</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">2. If you are not familiar with this document please send me an email. It is imperative that each one of my suppliers understands this document. As it can help you prepare your orders, and prepare for upcoming events.</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">3. This should be reviewed each time you receive it for changes to the forecast, and you should ensure you are aligned to the forecast<strong>. If you are not aligned reach out to me.</strong></span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">4. Should you have any other questions, please reach out to your RM.</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">Thank you,</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style="font-size:16px;">Walmart Replenishment</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><span style='font-size:19px;font-family:"Arial",sans-serif;'>&nbsp;</span></p>
<p style='margin:0in;font-size:15px;font-family:"Calibri",sans-serif;'><em><span style="font-size:13px;">Disclaimer: The information transmitted is intended only for the person or entity to which it is addressed and may contain proprietary, business-confidential and/or privileged material. If you are not the intended recipient of this message, you are hereby notified that any use, review, retransmission, dissemination, distribution, reproduction or any action taken in reliance upon this message is prohibited. If you received this in error, please contact the sender and delete the material from any computer.</span></em></p>
"""

In [0]:
# Read dataset to be used to create the email distribution list

# Ensure that the filename in the dataiku.Dataset() function contains
# the unique "Vendor_nbr", "to", "sender", and "cc" columns

dist_list_dss = dataiku.Dataset("dist_list")
dist_list = dist_list_dss.get_dataframe()

In [0]:
# Read full data dump

# Ensure that this dataset contains the "Vendor_nbr" column, which
# will be used to filter contents to be sent to each Vendor on the dist_list

dataset = dataiku.Dataset("data_dump_all")
df = dataset.get_dataframe()

In [0]:
# Static file attachments.  Hard coded for now, as of 2023-08-24

folder = dataiku.Folder("auipQAwL")
file1 = folder.get_path() + '/file_example_XLS_50.xls'
#file2 = folder.get_path() + '/<insertfile2name.'

In [0]:
# Establish schema for log of sent emails

output = dataiku.Dataset("dist_list_sent")
output_schema = [{'name': 'Vendor_nbr', 'type': 'string'},
                 {'name': 'sendmail_status', 'type': 'string'},
                 {'name': 'sendmail_error', 'type': 'string'}]
output.write_schema(output_schema)

In [0]:
# Loops through the dist_list for each unique "Vendor_nbr_8"
# - Creates a Subject line matching Vendor Name
# - Sets "to" email distribution list
# - Sets "sender" email (must equal SMTP server)
# - Sets "cc" email list (must equal SMTP server)
# - Filters the "full data dump" by vendor, attaches as CSV
# - Attaches static file1 and file2
# - Executes the mail_html function which actually sends the email/attachments

with output.get_writer() as writer:
    j = 0
    success = 0
    fail = 0
    try:
        for i in range(len(dist_list)):
            d = {}
            if i == 0: # Start a stopwatch the very first pass
                start = datetime.now()
            try:
                vn = str(dist_list['Vendor_nbr'][i])
                subject = 'Supplier ' + dist_list['Vndr_name'][i] + ' Vendor# ' + vn
                to = dist_list['SupplierContact'][i]
                sender = dist_list['RM_EMAIL'][i]
                cc = dist_list['RM_CC'][i]
                attach1 = df.loc[df['Vendor_nbr'] == dist_list['Vendor_nbr'][i]]
                attach1.to_csv('data_dump.csv', index = False)
                logging.info(f'Sending to {to}, Vendor {vn}')
                mail_html(subject=subject,
                          to=to,
                          sender=sender,
                          cc=cc,
                          html=html_text,
                          attachments=['data_dump.csv',file1])#,file2])
                end = datetime.now() #updates the stopwatch every pass
                rtime = random.randint(0, 4)
                time.sleep(rtime) #used to delay seconds between each email
                d['Vendor_nbr'] = str(dist_list['Vendor_nbr'][i])
                d['sendmail_status'] = 'SUCCESS'
                #print(d)
                success += 1
                if writer:
                    writer.write_row_dict(d)
            except Exception as e:
                logging.exception("Send failed")
                fail += 1
                d['Vendor_nbr'] = str(dist_list['Vendor_nbr'][i])
                d['sendmail_status'] = 'FAILED'
                d['sendmail_error'] = str(e)
                if writer:
                    writer.write_row_dict(d)
                end = datetime.now()
            j += 1
            if j % 10 == 0:
                logging.info(f'Sent {j} emails ({success} success {fail} fail)')
    except RuntimeError as runtime_error:
        logging.info(f'Exception {runtime_error}')
        end = datetime.now()
    elapsed = end - start
    logging.info(f'Sent {j} emails ({success} success {fail} fail)')
    logging.info(f'Time Elapsed: {elapsed}')